# CS6120 - Natural Language Processing - Summer2024
Instructor: Amir Tahmasebi

## Assignment 5 - Part I

# Language Models: Auto-Complete

In this assignment, you will build an auto-complete system.

A key building block for an auto-complete system is language models.
A Language models assigns the probability to a sequence of words, in a way that more "likely" sequences receives higher scores.  

While a variety of language models have been developed, this assignment uses **N-grams**, a simple but still powerful methodology for language modeling.
N-grams are used not only for auto-complete systems, but are also useful in machine translation, speech recognition, and could even help you classify authors given a text.


Here is the steps of this assignment:

1. Load and preprocess data
    - Load and tokenize data.
    - Split the sentences into training and test sets.
    - Replace words with a low frequency by an unknown marker '<unk>'.
1. Develop N-gram based language models
    - Compute the count of n-grams from a given data set.
    - Estimate the conditional probability of a next word with k-smoothing.
1. Evaluate the N-gram models by computing the perplexity score.
1. Use your own model to suggest an upcoming word given your sentence.

In [ ]:
import math
import random
import numpy as np
import pandas as pd
import nltk
nltk.data.path.append('.')

## Part 1: Load and Preprocess Data

We will be using twitter data for our exercise.
Let's load the data and see the first few sentences by the running the next cell.

Notice that data is a long string that contains many many tweats.
Observe that we have a linebreak "\n" between two sentences (tweats).

In [ ]:
with open("./en_US.twitter.txt", "r") as f: # update the path accordingly.
    data = f.read()
print("Data type:", type(data))
print("Number of letters:", len(data))
print("First 300 letters of the data")
print("-------")
display(data[0:300])
print("-------")

print("Last 300 letters of the data")
print("-------")
display(data[-300:])
print("-------")

Data type: <class 'str'>
Number of letters: 164456396
First 300 letters of the data
-------


"How are you? Btw thanks for the RT. You gonna be in DC anytime soon? Love to see you. Been way, way too long.\nWhen you meet someone special... you'll know. Your heart will beat more rapidly and you'll smile for no reason.\nthey've decided its more fun if I don't.\nSo Tired D; Played Lazer Tag & Ran A "

-------
Last 300 letters of the data
-------


'od. I see the success you got poppin in yo area.\nRT : Consumers are visual. They want data at their finger tips. Mobile is the only way to deliver this, 24/7.\nu welcome\nIt is #RHONJ time!!\nThe key to keeping your woman happy= attention, affection, treat her like a queen and sex her like a pornstar!\n'

-------


We will preprocess this data by the following steps:

1. Split data into sentences.
1. Split each sentence into tokens.
1. Split sentences into training and test data.
1. Find tokens that appear at least N times in the training data.
1. Replace tokens that appear less than N times by "<unk\>"

Note that in this assignment we use "token" and "words" interchangeably.

Note: we omit validation data in this exercise.
In real applications, we should hold a part of data as a validation set and use it to tune our training.
We skip this process for simplicity.

The first step of the data processing is to split this large string into sentences.  For simplicity, we will use the linebreak "\n" as the delimiter.

Hint: Use [str.split](https://docs.python.org/3/library/stdtypes.html?highlight=split#str.split) method.

In [ ]:
### To be completed: split_to_sentences ###
def split_to_sentences(data):
    """
    Split data by linebreak "\n"

    Args:
        data: str

    Returns:
        A list of sentences
    """
    sentences = []

    ### START CODE HERE ###
    sentences = data.split("\n")
    sentences = [s.strip() for s in sentences if s.strip() != '']
    ### END CODE HERE ###

    return sentences

In [ ]:
# test your code
x = """
I have a pen.\nI have an apple. \nAh\nApple pen.\n
"""
print(x)

split_to_sentences(x)


I have a pen.
I have an apple. 
Ah
Apple pen.




['I have a pen.', 'I have an apple.', 'Ah', 'Apple pen.']

Expected answer:
```
['I have a pen.', 'I have an apple.', 'Ah', 'Apple pen.']
```

The next step is to tokenize sentences, that is, we split a sentence into a sequence of tokens (words).
We also convert all tokens into lower cases not to distinguish the words appearing at the beginning.

Hint:
Use [str.lower](https://docs.python.org/3/library/stdtypes.html?highlight=split#str.lower) method for converting strings to lower cases.
For splitting a sentence into tokens, [nltk.word_tokenize](https://www.nltk.org/api/nltk.tokenize.html#nltk.tokenize.punkt.PunktLanguageVars.word_tokenize) function is helpful.(Implementation with [str.split](https://docs.python.org/3/library/stdtypes.html?highlight=split#str.split) would require some additional edge cases (e.g. comma or period right after a word).

In [ ]:
### To be completed: tokenize_sentences ###
nltk.download('punkt')
from nltk.tokenize import word_tokenize

def tokenize_sentences(sentences):
    """
    Tokenize sentences into tokens (words)

    Args:
        sentences: List of strings

    Returns:
        List of lists of tokens
    """

    tokenized_sentences = []

    ### START CODE HERE ###

    for sentence in sentences:
        tokens = word_tokenize(sentence)
        tokens = [token.lower() for token in tokens]
        tokenized_sentences.append(tokens)

    ### END CODE HERE ###

    return tokenized_sentences

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# test your code
sentences = ["Sky is blue.", "Leaves are green.", "Roses are red."]
tokenize_sentences(sentences)

[['sky', 'is', 'blue', '.'],
 ['leaves', 'are', 'green', '.'],
 ['roses', 'are', 'red', '.']]

Expected answer:

```
[['sky', 'is', 'blue', '.'],
 ['leaves', 'are', 'green', '.'],
 ['roses', 'are', 'red', '.']]
```

In [ ]:
# This is an example for the previous function:
def test_tokenize_sentences():
    test_cases = [
        {
            "name": "notebook example",
            "input": ["Sky is blue.", "Leaves are green.", "Roses are red."],
            "expected": [['sky', 'is', 'blue', '.'], ['leaves', 'are', 'green', '.'], ['roses', 'are', 'red', '.']]
        },
        {
            "name": "no sentence",
            "input": [],
            "expected": []
        },
        {
            "name": "one sentence with ;",
            "input": ["Grass is greener;"],
            "expected": [['grass', 'is', 'greener', ';']]
        },
        {
            "name": "two sentences, one in CAPS",
            "input": ["Space if infinite.", "OR IS IT?"],
            "expected": [['space', 'if', 'infinite', '.'], ['or', 'is', 'it', '?']]
        },
        {
            "name": "long sentence",
            "input": ["Really really long sentence. It is very long indeed; so long..."],
            "expected": [['really', 'really', 'long', 'sentence', '.', 'it', 'is', 'very', 'long', 'indeed', ';', 'so', 'long', '...']]
        }
    ]

    for test_case in test_cases:
        assert test_case["expected"] == tokenize_sentences(test_case["input"])
        print('Successfully passed test case: ' + test_case["name"])

test_tokenize_sentences()


Successfully passed test case: notebook example
Successfully passed test case: no sentence
Successfully passed test case: one sentence with ;
Successfully passed test case: two sentences, one in CAPS
Successfully passed test case: long sentence


As covered in the lecture, we won't be using all the tokens (words) appearing in the data for training.  Instead, we will use only the words that appear at least N times in the data.
To do so, let's count up all the words in the data set.

Hint: You need a double for-loop, one for sentences and the other for tokens within a sentence.

Using the function above, we can now preprare the tokenized data.

In [ ]:
### To be completed: get_tokenized_data ###
def get_tokenized_data(data):
    """
    Make a list of tokenized sentences

    Args:
        data: String

    Returns:
        List of lists of tokens
    """

    ### START CODE HERE ###
     # split data into sentences
    sentences = split_to_sentences(data)

    # tokenize sentences
    tokenized_sentences = tokenize_sentences(sentences)
    ### END CODE HERE ###

    return tokenized_sentences

In [ ]:
x = "Sky is blue.\nLeaves are green\nRoses are red."
get_tokenized_data(x)

[['sky', 'is', 'blue', '.'],
 ['leaves', 'are', 'green'],
 ['roses', 'are', 'red', '.']]

Expected outcome:

```
[['sky', 'is', 'blue', '.'],
 ['leaves', 'are', 'green'],
 ['roses', 'are', 'red', '.']]
```

Now run the cell below to split data into training and test sets.

In [ ]:
tokenized_data = get_tokenized_data(data)
random.seed(87)
random.shuffle(tokenized_data)

train_size = int(len(tokenized_data) * 0.8)
train_data = tokenized_data[0:train_size]
test_data = tokenized_data[train_size:]

In [ ]:
print("{} data are split into {} train and {} test set".format(
    len(tokenized_data), len(train_data), len(test_data)))

print("First training sample:")
print(train_data[0])

print("First test sample")
print(test_data[0])

2360148 data are split into 1888118 train and 472030 test set
First training sample:
['adam', 'sandler', 'is', 'not', 'being', 'funny', 'on', '#', 'peoplechoice', '.', 'make', 'it', 'stop']
First test sample
['“', ':', 'dwayne', 'wade', 'accused', 'pacers', 'of', 'excessive', 'celebration', '.', 'the', 'heat', 'would', 'never', 'celebrate', 'before', 'they', 'won', 'anything', '.', '”']


One problem you can bump into when using n-grams for predicting the upcoming word is what happens if you input a word that you have never seen in the dataset. These are called unknown words, or <b>out of vocabulary (OOV)</b> words. The percentage of unknown words in the test set is called the <b> OOV </b> rate. In such a case, you will not be able to predict the upcoming word because you do not have any counts for that specific word. To mitigate this problem, you would look at all the words in the training set, and only keep a list of the most frequent words. This will be your <b> closed vocabulary </b>. All the other words that are not part of your closed vocabulary will be changed to the word 'unk'. You will now create a function that takes in a text document, n, which is the number of most frequent words used that you want to keep, returns the document containing only the word closed vocabulary and the word unk.

In [ ]:
### To be completed: count_words ###
def count_words(tokenized_sentences):
    """
    Count the number of word appearence in the tokenized sentences

    Args:
        tokenized_sentences: List of lists of strings

    Returns:
        dict that maps word (str) to the frequency (int)
    """
    word_counts = {}

    ### START CODE HERE ###
    for sentence in tokenized_sentences:
        for token in sentence:
            word_counts[token] = word_counts.get(token, 0) + 1
    ### END CODE HERE ###

    return word_counts

In [ ]:
# test your code
tokenized_sentences = [['sky', 'is', 'blue', '.'],
                       ['leaves', 'are', 'green', '.'],
                       ['roses', 'are', 'red', '.']]
count_words(tokenized_sentences)

{'sky': 1,
 'is': 1,
 'blue': 1,
 '.': 3,
 'leaves': 1,
 'are': 2,
 'green': 1,
 'roses': 1,
 'red': 1}

Expected answer (order may differ):

```
{'sky': 1,
 'is': 1,
 'blue': 1,
 '.': 3,
 'leaves': 1,
 'are': 2,
 'green': 1,
 'roses': 1,
 'red': 1}
```

Using the word counts computed above, we will now find the N most frequence words.
The implementation requires a bit of care, since dictionary type is essentially

In [ ]:
### To be completed: get_words_with_nplus_frequency ###
def get_words_with_nplus_frequency(tokenized_sentences, minimum_freq):
    """
    Find the words that appear N times or more

    Args:
        tokenized_sentences: List of lists of sentences
        minimum_freq: Minimum frequency, i.e. N

    Returns:
        List of words that appear N times or more
    """
    filtered_words = []
    word_counts = count_words(tokenized_sentences)

    ### START CODE HERE ###
    filtered_words = [word for word, count in word_counts.items() if count >= minimum_freq]

    ### END CODE HERE ###

    return filtered_words

In [ ]:
# test your code
tokenized_sentences = [['sky', 'is', 'blue', '.'],
                       ['leaves', 'are', 'green', '.'],
                       ['roses', 'are', 'red', '.']]
get_words_with_nplus_frequency(tokenized_sentences, 2)

['.', 'are']

Expected answer:

```
['.', 'are']
```

The words that appear n times or more are treated as our "vocabulary" and all other words are regarded as unknown.
Hence, we will replace words not in our vocabulary by a marker "<unk\>".

In [ ]:
### To be completed: replace_oov_words_by_unk ###
def replace_oov_words_by_unk(tokenized_sentences, vocabulary, unknown_marker="<unk>"):
    """
    Replace words not in the given vocabulary by unknown marker

    Args:
        tokenized_sentences: List of lists of strings
        vocabulary: List of strings that we will use
        unknown_marker: A string representing unknown (out-of-vocabulary) words

    Returns:
        List of lists of strings, with words not in the vocabulary replaced
    """
    vocabulary = set(vocabulary)  # this makes each search faster
    replaced_tokenized_sentences = []
    for sentence in tokenized_sentences:
        replaced_sentence = [
            word if word in vocabulary else unknown_marker
            for word in sentence
        ]

        ### START CODE HERE ###

        ### END CODE HERE ###

        replaced_tokenized_sentences.append(replaced_sentence)
    return replaced_tokenized_sentences

In [ ]:
tokenized_sentences = [["dogs", "run"], ["cats", "sleep"]]
vocabulary = ["dogs", "sleep"]
replace_oov_words_by_unk(tokenized_sentences, vocabulary)

[['dogs', '<unk>'], ['<unk>', 'sleep']]

Expected answer:

```
['dogs', '<unk>'], ['<unk>', 'sleep']]
```

Now we are ready to process our data combining the functions defined above.
Specifically, we will

1. Find tokens that appear at least N times in the training data.
1. Replace tokens that appear less than N times by "<unk\>" both for training and test data.

In [ ]:
### To be completed: preprocess_data ###
def preprocess_data(train_data, test_data, minimum_freq):
    """
    Preprocess data, i.e.,
        - Find tokens that appear at least N times in the training data.
        - Replace tokens that appear less than N times by "<unk>" both for training and test data.
    Args:
        train_data, test_data: List of lists of strings.
        minimum_freq: Minimum frequency. Words that appear less than this times will be
                      treated as unknown.

    Returns:
        Tuple of
        - training data with low frequent words replaced by "<unk>"
        - test data with low frequent words replaced by "<unk>"
        - vocabulary of words that appear n times or more in the training data
    """
    ### START CODE HERE ###
    vocabulary = get_words_with_nplus_frequency(train_data, minimum_freq)

    # Replace OOV words by <unk>
    train_data_replaced = replace_oov_words_by_unk(train_data, vocabulary)
    test_data_replaced = replace_oov_words_by_unk(test_data, vocabulary)
    ### END CODE HERE ###

    return train_data_replaced, test_data_replaced, vocabulary

In [ ]:
# test your code
x = [['sky', 'is', 'blue', '.'],
     ['leaves', 'are', 'green']]
y = [['roses', 'are', 'red', '.']]

preprocess_data(x, y, 1)

([['sky', 'is', 'blue', '.'], ['leaves', 'are', 'green']],
 [['<unk>', 'are', '<unk>', '.']],
 ['sky', 'is', 'blue', '.', 'leaves', 'are', 'green'])

Expected outcome:

```
([['sky', 'is', 'blue', '.'], ['leaves', 'are', 'green']],
 [['<unk>', 'are', '<unk>', '.']],
 ['sky', 'is', 'blue', '.', 'leaves', 'are', 'green'])
```

Finally, run the cell below to complete the preprocessing both for training and test sets.

In [ ]:
minimum_freq = 2
train_data_processed, test_data_processed, vocabulary = preprocess_data(
    train_data, test_data, minimum_freq)

In [ ]:
print("First preprocessed training sample:")
print(train_data_processed[0])

print("First preprocessed test sample:")
print(test_data_processed[0])

print("First 10 vocabulary:")
print(vocabulary[0:10])

print("Size of vocabulary:", len(vocabulary))

First preprocessed training sample:
['adam', 'sandler', 'is', 'not', 'being', 'funny', 'on', '#', '<unk>', '.', 'make', 'it', 'stop']
First preprocessed test sample:
['“', ':', 'dwayne', 'wade', 'accused', 'pacers', 'of', 'excessive', 'celebration', '.', 'the', 'heat', 'would', 'never', 'celebrate', 'before', 'they', 'won', 'anything', '.', '”']
First 10 vocabulary:
['adam', 'sandler', 'is', 'not', 'being', 'funny', 'on', '#', '.', 'make']
Size of vocabulary: 145637


You are done with the preprocessing section of the assignment.
Objects `train_data_processed`, `test_data_processed`, are `vocabulary` to be used in the rest of the exercises.

## Part 2: Develop n-gram based language models

In this section, we will develop language models using the data prepared in the previous section.
Our modelling strategy is **n-grams**; we assume the probability of the next word depends only on the previous n-gram (i.e., $n$ words).
Mathematically, we can write the conditional probability for the t-th word as:

$$ P(w_t | w_{t-1}\dots w_{t-n}) \tag{1}$$

where $w_t$ denotes the $t$-th word in a sentence.

We will estimate this probability from the training data.
A natural estimator for the probability is the fraction of counts:

$$ \hat{P}(w_t | w_{t-1}\dots w_{t-n}) = \frac{C(w_{t-1}\dots w_{t-n}, w_n)}{C(w_{t-1}\dots w_{t-n})} \tag{2} $$

where $C(\cdots)$ denotes the number of occurence of the given sequence. $\hat{P}$ means the estimation of $P$. Notice that denominator of the equation (2) is the number of occurence of the previous $n$ words, and the numerator is the same sequence followed by the word $w_t$.

As covered in the lecture, we will later modify the equation (2) by adding k-smoothing, which avoid errors in zero count cases.

The equation (2) tells us that to estimate probabilities based on n-grams, we need the counts of n-grams (for denominator) and (n+1)-grams (for numerator).
So, in the next cell, you will implement a function that computes the counts of n-grams for an arbitrary $n$.

When computing the counts for n-grams, we prepend $n-1$ starting markers "<s\>" to indicate the beginning of the sentence.  For example, in the bi-gram model, a sequence "<s\><s\>" should predict the first word of a sentence.
We also append a ending marker "<e\>" so that the model can predict when to finish a sentence.

Technical note: In this implementation, we will store the counts as a dictionary that maps a tuple of n words to the number of occurence.  The reason why we use tuple instead of list is because a list in Python is mutable object, and hence it is not intended to be used as a key to a dictionary.

In [ ]:
### To be completed: count_n_grams ###
def count_n_grams(data, n):
    """
    Count all n-grams in the data

    Args:
        data: List of lists of words
        n: number of words in a sequence

    Returns:
        A dictionary that maps a tuple of n-words to its frequency
    """
    n_grams = {}
    for sentence in data:
        # prepend <s> n times and append <s>
        sentence = ["<s>"]*n + sentence + ["<e>"]
        # convert list to tuple for using sequence as dictionary keys
        sentence = tuple(sentence)
        for i in range(len(sentence) - n + 1):

            ### START CODE HERE ###
            n_gram = sentence[i:i+n]

            n_grams[n_gram] = n_grams.get(n_gram, 0) + 1
            ### END CODE HERE ###

    return n_grams

In [ ]:
# test your code
# Outcome does not match expected outcome
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
print("Uni-gram:")
print(count_n_grams(sentences, 1))
print("Bi-gram:")
print(count_n_grams(sentences, 2))

Uni-gram:
{('<s>',): 2, ('i',): 1, ('like',): 2, ('a',): 2, ('cat',): 2, ('<e>',): 2, ('this',): 1, ('dog',): 1, ('is',): 1}
Bi-gram:
{('<s>', '<s>'): 2, ('<s>', 'i'): 1, ('i', 'like'): 1, ('like', 'a'): 2, ('a', 'cat'): 2, ('cat', '<e>'): 2, ('<s>', 'this'): 1, ('this', 'dog'): 1, ('dog', 'is'): 1, ('is', 'like'): 1}


Expected outcome:

```
Uni-gram:
{('<s>',): 2, ('i',): 1, ('like',): 2, ('a',): 2, ('cat',): 2, ('<e>',): 2, ('this',): 1, ('dog',): 1, ('is',): 1}
Bi-gram:
{('<s>', '<s>'): 2, ('<s>', 'i'): 1, ('i', 'like'): 1, ('like', 'a'): 2, ('a', 'cat'): 2, ('cat', '<e>'): 2, ('<s>', 'this'): 1, ('this', 'dog'): 1, ('dog', 'is'): 1, ('is', 'like'): 1}
```

Next, we estimate the probability using the n-gram counts.
We show the equation (2) again here:

$$ \hat{P}(w_t | w_{t-1}\dots w_{t-n}) = \frac{C(w_{t-1}\dots w_{t-n}, w_n)}{C(w_{t-1}\dots w_{t-n})} \tag{2} $$

While this is a natural estimator, it has limitation in dealing with zero count cases.
Suppose we encounter an n-gram that did not occur in the training data.  Then, the equation (2) cannot be evaluated (it becomes zero divided by zero).
A way to avoid such case is to add k-smoothing.  In effect, it adds a positive constant $k$ to each numerator counts in the equation (2):

$$ \hat{P}(w_t | w_{t-1}\dots w_{t-n}) = \frac{C(w_{t-1}\dots w_{t-n}, w_n) + k}{C(w_{t-1}\dots w_{t-n}) + k|V|} \tag{3} $$

where $|V|$ is the size of vocabulary.

For unseen n-grams, the equation (3) becomes $1/|V|$, i.e. picks all words with the same probability.

In the next cell, you will define a function that computes the probability estimate (3) from n-gram counts and a constant $k$.

Hint: In the imprementation, we use pre-computed n-gram and (n+1)-gram counts for calculating the equation (3).

In [ ]:
### To be completed: estimate_probabilityy ###
def estimate_probability(word, previous_n_gram,
                         n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=1.0):
    """
    Estimate the probabilities of a next word using the n-gram counts with k-smoothing

    Args:
        word: next word
        previous_n_gram: A sequence of words of length n
        n_gram_counts: Dictionary of counts of (n+1)-grams
        n_plus1_gram_counts: Dictionary of counts of (n+1)-grams
        vocabulary_size: number of words in the vocabulary
        k: positive constant, smoothing parameter

    Returns:
        A probability
    """
    previous_n_gram = tuple(previous_n_gram)

    ### START CODE HERE ###
    previous_n_gram_count = n_gram_counts.get(previous_n_gram, 0)
    denominator = previous_n_gram_count + k * vocabulary_size

    n_plus1_gram = previous_n_gram + (word,)
    n_plus1_gram_count = n_plus1_gram_counts.get(n_plus1_gram, 0)

    numerator = n_plus1_gram_count + k
    probability=numerator/denominator

    ### END CODE HERE ###

    return probability

In [ ]:
# test your code
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)
estimate_probability("cat", "a", unigram_counts, bigram_counts, len(unique_words), k=1)

0.3333333333333333

Expected outcome:

```
0.3333333333333333
```

The function defined below loops over all words in vocabulary to calculate probabilities for all possible words.

In [ ]:
def estimate_probabilities(previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0):
    """
    Estimate the probabilities of next words using the n-gram counts with k-smoothing

    Args:
        previous_n_gram: A sequence of words of length n
        n_gram_counts: Dictionary of counts of (n+1)-grams
        n_plus1_gram_counts: Dictionary of counts of (n+1)-grams
        vocabulary: List of words
        k: positive constant, smoothing parameter

    Returns:
        A dictionary mapping from next words to the probability.
    """
    # convert list to tuple to use it as a dictionary key
    previous_n_gram = tuple(previous_n_gram)

    # add <e> <unk> to the vocabulary
    # <s> is not needed since it should not appear as the next word
    vocabulary = vocabulary + ["<e>", "<unk>"]
    vocabulary_size = len(vocabulary)

    probabilities = {}
    for word in vocabulary:
        probability = estimate_probability(word, previous_n_gram,
                                           n_gram_counts, n_plus1_gram_counts,
                                           vocabulary_size, k=k)
        probabilities[word] = probability

    return probabilities

In [ ]:
# test your code
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))
unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)
estimate_probabilities("a", unigram_counts, bigram_counts, unique_words, k=1)

{'like': 0.09090909090909091,
 'a': 0.09090909090909091,
 'is': 0.09090909090909091,
 'i': 0.09090909090909091,
 'cat': 0.2727272727272727,
 'this': 0.09090909090909091,
 'dog': 0.09090909090909091,
 '<e>': 0.09090909090909091,
 '<unk>': 0.09090909090909091}

Expected outcome:

```
{'is': 0.09090909090909091,
 'like': 0.09090909090909091,
 'dog': 0.09090909090909091,
 'cat': 0.2727272727272727,
 'i': 0.09090909090909091,
 'a': 0.09090909090909091,
 'this': 0.09090909090909091,
 '<e>': 0.09090909090909091,
 '<unk>': 0.09090909090909091}
```

In [ ]:
trigram_counts = count_n_grams(sentences, 3)
estimate_probabilities(["<s>", "<s>"], bigram_counts, trigram_counts, unique_words, k=1)

{'like': 0.09090909090909091,
 'a': 0.09090909090909091,
 'is': 0.09090909090909091,
 'i': 0.18181818181818182,
 'cat': 0.09090909090909091,
 'this': 0.18181818181818182,
 'dog': 0.09090909090909091,
 '<e>': 0.09090909090909091,
 '<unk>': 0.09090909090909091}

Expected outcome:

```
{'is': 0.09090909090909091,
 'like': 0.09090909090909091,
 'dog': 0.09090909090909091,
 'cat': 0.09090909090909091,
 'i': 0.18181818181818182,
 'a': 0.09090909090909091,
 'this': 0.18181818181818182,
 '<e>': 0.09090909090909091,
 '<unk>': 0.09090909090909091}
```

### Count and probability matrices

As we have seen so far, the n-gram counts computed above are sufficient for computing the probabilities of the nex words.  Yet, it might be more intuitive to present them as count or probability matrices that we covered in the lecture.
The functions defined in the next cells return count or probability matrices.

In [ ]:
def make_count_matrix(n_plus1_gram_counts, vocabulary):
    # add <e> <unk> to the vocabulary
    # <s> is omitted since it should not appear as the next word
    vocabulary = vocabulary + ["<e>", "<unk>"]

    # obtain unique n-grams
    n_grams = []
    for n_plus1_gram in n_plus1_gram_counts.keys():
        n_gram = n_plus1_gram[0:-1]
        n_grams.append(n_gram)
    n_grams = list(set(n_grams))

    # mapping from n-gram to row
    row_index = {n_gram:i for i, n_gram in enumerate(n_grams)}
    # mapping from next word to column
    col_index = {word:j for j, word in enumerate(vocabulary)}

    nrow = len(n_grams)
    ncol = len(vocabulary)
    count_matrix = np.zeros((nrow, ncol))
    for n_plus1_gram, count in n_plus1_gram_counts.items():
        n_gram = n_plus1_gram[0:-1]
        word = n_plus1_gram[-1]
        if word not in vocabulary:
            continue
        i = row_index[n_gram]
        j = col_index[word]
        count_matrix[i, j] = count

    count_matrix = pd.DataFrame(count_matrix, index=n_grams, columns=vocabulary)
    return count_matrix

In [ ]:
sentences = [['i', 'like', 'a', 'cat'],
                 ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))
bigram_counts = count_n_grams(sentences, 2)
display(make_count_matrix(bigram_counts, unique_words))

trigram_counts = count_n_grams(sentences, 3)
display(make_count_matrix(trigram_counts, unique_words))

,like,a,is,i,cat,this,dog,<e>,<unk>
"(<s>,)",0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
"(this,)",0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
"(is,)",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(cat,)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
"(dog,)",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
"(a,)",0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
"(like,)",0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(i,)",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,like,a,is,i,cat,this,dog,<e>,<unk>
"(<s>, this)",0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
"(this, dog)",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
"(like, a)",0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
"(a, cat)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
"(dog, is)",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(is, like)",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(i, like)",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(<s>, i)",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(<s>, <s>)",0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


In [ ]:
def make_probability_matrix(n_plus1_gram_counts, vocabulary, k):
    count_matrix = make_count_matrix(n_plus1_gram_counts, unique_words)
    count_matrix += k
    prob_matrix = count_matrix.div(count_matrix.sum(axis=1), axis=0)
    return prob_matrix

In [ ]:
sentences = [['i', 'like', 'a', 'cat'],
                 ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))
bigram_counts = count_n_grams(sentences, 2)
display(make_probability_matrix(bigram_counts, unique_words, k=1))

trigram_counts = count_n_grams(sentences, 3)
display(make_probability_matrix(trigram_counts, unique_words, k=1))

,like,a,is,i,cat,this,dog,<e>,<unk>
"(<s>,)",0.090909,0.090909,0.090909,0.181818,0.090909,0.181818,0.090909,0.090909,0.090909
"(this,)",0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.200000,0.100000,0.100000
"(is,)",0.200000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
"(cat,)",0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.272727,0.090909
"(dog,)",0.100000,0.100000,0.200000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
"(a,)",0.090909,0.090909,0.090909,0.090909,0.272727,0.090909,0.090909,0.090909,0.090909
"(like,)",0.090909,0.272727,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909
"(i,)",0.200000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000


,like,a,is,i,cat,this,dog,<e>,<unk>
"(<s>, this)",0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.200000,0.100000,0.100000
"(this, dog)",0.100000,0.100000,0.200000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
"(like, a)",0.090909,0.090909,0.090909,0.090909,0.272727,0.090909,0.090909,0.090909,0.090909
"(a, cat)",0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.272727,0.090909
"(dog, is)",0.200000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
"(is, like)",0.100000,0.200000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
"(i, like)",0.100000,0.200000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
"(<s>, i)",0.200000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
"(<s>, <s>)",0.090909,0.090909,0.090909,0.181818,0.090909,0.181818,0.090909,0.090909,0.090909


Confirm that we obtain the same results as for the `estimate_probabilities` function above.

## Part 3: Perplexity

In this section, you will generate the perplexity score to evaluate your model on the test set. You will also use back-off when needed. Perplexity is used as an evaluation metric of your language model. If you were to calculate the  the perplexity score of your test set, on a bigram model, you compute the following formula:
$$ PP(W) =\sqrt[N]{ \prod_{t=1}^N \frac{1}{P(w_t | w_{t-1})} } \tag{4}$$

where $N$ is the length of the sentence.

The higher our probabilities are the lower the perplexity will be. The more our n-grams tell us about the sentence the lower our perplexity score will be.

</b>
<b>Instructions:</b> Compute the perplexity score given an N-gram count matrix and a sentence.

In [ ]:
# To be completed: calculate_perplexity
def calculate_perplexity(sentence, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=1.0):
    """
    Calculate perplexity for a list of sentences

    Args:
        sentence: List of strings
        n_gram_counts: Dictionary of counts of (n+1)-grams
        n_plus1_gram_counts: Dictionary of counts of (n+1)-grams
        vocabulary_size: number of unique words in the vocabulary
        k: Positive smoothing constant

    Returns:
        Perplexity score
    """
    n = len(list(n_gram_counts.keys())[0])
    sentence = ["<s>"] * n + sentence + ["<e>"]
    sentence = tuple(sentence)
    N = len(sentence)
    p = 1.0

    for i in range(N - n):

        ### START CODE HERE ###
        n_gram = sentence[i:i+n]
        word = sentence[i+n]
        probability = estimate_probability(word, n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k)
        p *= 1 / probability
        ### END CODE HERE ###

    perplexity = (p)**(1/N)

    return perplexity

In [ ]:
# test your code

sentences = [['i', 'like', 'a', 'cat'],
                 ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)


perplexity_train1 = calculate_perplexity(sentences[0],
                                         unigram_counts, bigram_counts,
                                         len(unique_words), k=1.0)
print("Perplexity for first train sample:", perplexity_train1)

test_sentence = ['i', 'like', 'a', 'dog']
perplexity_test = calculate_perplexity(test_sentence,
                                       unigram_counts, bigram_counts,
                                       len(unique_words), k=1.0)
print("Perplexity for test sample:", perplexity_test)

Perplexity for first train sample: 2.8039657955522013
Perplexity for test sample: 3.965406456500188


Expected Output:

```
2.8039657955522013
3.965406456500188
```

<b> Note: </b> If your sentence is really long, then you take the sum of the log of the probabilities to avoid underflow errors, instead of multiplying.

## Part 4: Develop auto-complete system

In this section, we will combine the language models developed so far to finally implement an auto-complete system.
In this implementation, we compute probabilities for all possible next word and suggest the most likely one.
This function also take an optional argument `start_with`, which specifies the first a few letters of the next words.

Hint: `estimate_probabilities` returns a mapping from word to the probability.

In [ ]:
# To be completed: suggest_a_word
def suggest_a_word(previous_tokens, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0, start_with=None):
    """
    Get suggestion for the next word

    Args:
        previous_tokens: The sentence you input where each token is a word. Must have length > n
        n_gram_counts: Dictionary of counts of (n+1)-grams
        n_plus1_gram_counts: Dictionary of counts of (n+1)-grams
        vocabulary: List of words
        k: positive constant, smoothing parameter
        start_with: If not None, specifies the first few letters of the next word

    Returns:
        A tuple of
          - string of the most likely next word
          - corresponding probability
    """
    n = len(list(n_gram_counts.keys())[0])
    previous_n_gram = previous_tokens[-n:]

    probabilities = estimate_probabilities(previous_n_gram,
                                           n_gram_counts, n_plus1_gram_counts,
                                           vocabulary, k=k)
    suggestion = None
    max_prob = 0

    ### START CODE HERE ###
    for word, prob in probabilities.items():
        if start_with:
            if not word.startswith(start_with):
                continue
        if prob > max_prob:
            suggestion = word
            max_prob = prob
    ### END CODE HERE

    return suggestion, max_prob

In [ ]:
# test your code
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)

previous_tokens = ["i", "like"]
suggest_a_word(previous_tokens, unigram_counts, bigram_counts, unique_words, k=1.0)

('a', 0.2727272727272727)

In [ ]:
# test your code
suggest_a_word(previous_tokens, unigram_counts, bigram_counts, unique_words, k=1.0, start_with="c")

('cat', 0.09090909090909091)

Expected outcomes:

```
('a', 0.2727272727272727)
('cat', 0.09090909090909091)
```

The function defined below loop over varioud n-gram models to get multiple suggestions.

In [ ]:
def get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0, start_with=None):
    model_counts = len(n_gram_counts_list)
    suggestions = []
    for i in range(model_counts-1):
        n_gram_counts = n_gram_counts_list[i]
        n_plus1_gram_counts = n_gram_counts_list[i+1]

        suggestion = suggest_a_word(previous_tokens, n_gram_counts,
                                    n_plus1_gram_counts, vocabulary,
                                    k=k, start_with=start_with)
        suggestions.append(suggestion)
    return suggestions

In [ ]:
# test your code
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)
trigram_counts = count_n_grams(sentences, 3)
quadgram_counts = count_n_grams(sentences, 4)
qintgram_counts = count_n_grams(sentences, 5)

n_gram_counts_list = [unigram_counts, bigram_counts, trigram_counts, quadgram_counts, qintgram_counts]
previous_tokens = ["i", "like"]
get_suggestions(previous_tokens, n_gram_counts_list, unique_words, k=1.0)

[('a', 0.2727272727272727),
 ('a', 0.2),
 ('like', 0.1111111111111111),
 ('like', 0.1111111111111111)]

## Show time!

Conguraturations!  You have developed all building blocks for implementing your own auto-complete systems.
Let's see this in action.

In [ ]:
n_gram_counts_list = []
for n in range(1, 5):
    print("Computing n-gram counts with n =", n, "...")
    n_model_counts = count_n_grams(train_data_processed, n)
    n_gram_counts_list.append(n_model_counts)

Computing n-gram counts with n = 1 ...
Computing n-gram counts with n = 2 ...
Computing n-gram counts with n = 3 ...
Computing n-gram counts with n = 4 ...


In [ ]:
previous_tokens = ["i", "am", "to"]
get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0)

[('be', 0.04837541210334448),
 ('see', 0.00012335949011410752),
 ('have', 8.233163181294254e-05)]

In [ ]:
previous_tokens = ["i", "want", "to", "go"]
get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0)

[('to', 0.05459580705775871),
 ('to', 0.02197760913667443),
 ('to', 0.003808591093337686)]

In [ ]:
previous_tokens = ["hey", "how", "are"]
get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0)

[('you', 0.06695038443273968),
 ('you', 0.017145501988905062),
 ('you', 0.0004872023605297468)]

In [ ]:
previous_tokens = ["hey", "how", "are", "you"]
get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0)

[("'re", 0.04808769761738585),
 ('?', 0.012814809849671252),
 ('?', 0.007976031418276044)]

In [ ]:
previous_tokens = ["hey", "how", "are", "you"]
get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0, start_with="d")

[('do', 0.01782318326471285),
 ('doing', 0.006873564198967746),
 ('doing', 0.0017881965531667948)]